In [45]:
using DifferentialEquations
using HydroTools
using Plots

# Example functions for D(θ) and K(θ)
D(θ) = 1.0 + θ^2  # Example non-linear diffusion coefficient
K(θ) = 0.1 * θ    # Example linear advection coefficient

# Initial conditions and parameters
u0 = fill(0.1, 100) |> collect # Example initial soil moisture profile
tspan = (0.0, 10.0)  # Time span for the simulation
# params = (dz = 0.1, D = D, K = K)  # Parameters including spatial discretization and functions

(dz = 0.1, D = D, K = K)

In [49]:
# Function to calculate hydraulic conductivity from water content
function van_genuchten_K(θ; param)
    (; θs, θr, Ks, α, n, m) = param
    # m = 1 - 1/n
    Se = (θ - θr) / (θs - θr)
    effective_saturation = Se^0.5
    term = (1 - (1 - Se^(1/m))^m)^2
    return Ks * effective_saturation * term
end

# Function to calculate pressure head psi from water content
function van_genuchten_ψ(θ; param)
    (; θs, θr, α, n, m) = param
    # m = 1 - 1/n
    if θ <= θr
        return Inf  # Return a very high positive number indicating very dry conditions
    elseif θ >= θs
        return 0  # Saturated condition, psi is zero
    else
        return (1 / α) * (((θs - θr) / (θ - θr))^(1/m) - 1)^(1/n)
    end
end

# van Genuchten parameters
θs = 0.287  # Saturated water content
θr = 0.075  # Residual water content
α = 0.027  # 1/cm
n = 3.96
m = 1
Ks=34 / 3600
param = (; θs, θr, Ks, α, n, m)

θ0 = 0.267
ψ0 = van_genuchten_ψ(θ0; param)

20.921265200082487

In [58]:
n = 150
dz = ones(n)
z, z₊ₕ, dz₊ₕ = soil_depth_init(dz)

dt = 5
ntim = 0.8 * 3600 / dt
p = (; dz, z, dt, K = zeros(n), ψ = zeros(n), θ0, ψ0, param)

(dz = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], z = [-0.5, -1.5, -2.5, -3.5, -4.5, -5.5, -6.5, -7.5, -8.5, -9.5  …  -140.5, -141.5, -142.5, -143.5, -144.5, -145.5, -146.5, -147.5, -148.5, -149.5], dt = 5, K = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], ψ = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], θ0 = 0.267, ψ0 = 20.921265200082487, param = (θs = 0.287, θr = 0.075, Ks = 0.009444444444444445, α = 0.027, n = 3.96, m = 1))

In [52]:
# z₊ₕ
# param
# van_genuchten_K.(u0; param)
van_genuchten_ψ.(u0; param)
    

100-element Vector{Float64}:
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
  ⋮
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255
 61.562813229655255

In [59]:
# Define the function representing the system of ODEs for soil moisture transport
function cal_Q(i, K, ψ, z)
    K₊ₕ_up = (K[i] + K[i-1]) / 2
    z₊ₕ_up = (z[i] + z[i-1]) / 2
    Q = -K₊ₕ_up * ( (ψ[i-1] - ψ[i]) / z₊ₕ_up + 1)
    Q, z₊ₕ_up
end

function soil_moisture_transport(du, u, p, t)
    n = length(u)
    (; dz, dt, z, K, ψ, θ0, ψ0, param) = p
    
    K .= van_genuchten_K.(u; param)
    ψ .= van_genuchten_ψ.(u; param)
    
    # Compute fluxes and update soil moisture
    # @inbounds 
    for i in 2:n-1
        Q_up, z₊ₕ_up     = cal_Q(i, K, ψ, z)
        Q_down, z₊ₕ_down = cal_Q(i+1, K, ψ, z)
        Δz = z₊ₕ_up - z₊ₕ_down
        du[i] = -(Q_up - Q_down) / Δz
    end
    
    ## boundary
    i = 1
    K₊ₕ_up = K[1]
    Q_up, z₊ₕ_up = -K₊ₕ_up * ( (ψ0 - ψ[i]) / z₊ₕ_up + 1), 0
    Q_down, z₊ₕ_down = cal_Q(i+1, K, ψ, z)
    
    Δz = z₊ₕ_up - z₊ₕ_down
    du[i] = -(Q_up - Q_down) / Δz
    
    i = n
    Q_up, z₊ₕ_up = cal_Q(i, K, ψ, z)
    Q_down = -K[i]
    Δz = (z[i-1] - z[i])/2
    du[i] = -(Q_up - Q_down) / Δz
end

# K₊ₕ_up = (K[i] + K[i-1]) / 2
# z₊ₕ_up = (z[i] + z[i-1]) / 2
# z₊ₕ_down = (z[i+1] + z[i]) / 2
# Q_up = -K₊ₕ_up * ( (ψ[i-1] - ψ[i]) / z₊ₕ_up + 1)
# K₊ₕ_down = (K[i+1] + K[i]) / 2
# Q_down = -K₊ₕ_down * ( (ψ[i] - ψ[i+1]) / z₊ₕ_down + 1)
# Δz = (z[i-1] - z[i+1])/2

soil_moisture_transport (generic function with 1 method)

20.921265200082487

In [3]:
# Define the function representing the system of ODEs
function richards_ode(du, u, p, t)
    n = length(u)
    dz = p.dz  # Spatial step size
    D, K = p.D, p.K  # Functions for diffusion and advection terms

    # Interior points
    @inbounds for i in 2:n-1
        # Non-linear diffusion term
        D_mid = (D(u[i+1]) + D(u[i])) / 2
        D_back = (D(u[i]) + D(u[i-1])) / 2
        diffusion = (D_mid * (u[i+1] - u[i]) - D_back * (u[i] - u[i-1])) / dz^2
        
        # Advection term
        K_mid = (K(u[i+1]) + K(u[i])) / 2
        K_back = (K(u[i]) + K(u[i-1])) / 2
        advection = (K_mid - K_back) / dz

        # Update rule for the ith point
        du[i] = diffusion + advection
    end
    
    # Boundary conditions can be more specific depending on the physical situation
    du[1] = du[2]  # Example: Neumann boundary condition at the top
    du[n] = du[n-1]  # Example: Neumann boundary condition at the bottom
end

richards_ode (generic function with 1 method)

In [61]:
# p.z

In [62]:
prob = ODEProblem(soil_moisture_transport, u0, tspan, p)
@time sol = solve(prob, Tsit5(), reltol=1e-6, abstol=1e-6, saveat=1)  # Using an adaptive solver suitable for stiff problems

# plot(sol)

LoadError: DimensionMismatch: array could not be broadcast to match destination

In [12]:
sol.u[1]

100-element Vector{Float64}:
 0.0
 0.010101010101010102
 0.020202020202020204
 0.030303030303030304
 0.04040404040404041
 0.050505050505050504
 0.06060606060606061
 0.0707070707070707
 0.08080808080808081
 0.09090909090909091
 0.10101010101010101
 0.1111111111111111
 0.12121212121212122
 ⋮
 0.8888888888888888
 0.898989898989899
 0.9090909090909091
 0.9191919191919192
 0.9292929292929293
 0.9393939393939394
 0.9494949494949495
 0.9595959595959596
 0.9696969696969697
 0.9797979797979798
 0.98989898989899
 1.0